In [ ]:
!pip install python-dotenv
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login

login()

In [ ]:
# # If HF token is stored in .env:
# import os
# os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')

## Load the Model and Tokenizer from Local

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Specify the path to your mer ged model directory
# merged_model_path = "merged_model"

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(merged_model_path)

# # Load the merged model
# model = AutoModelForCausalLM.from_pretrained(
#     merged_model_path,
#     torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
#     device_map="auto"  # Automatically maps model to available devices
# )

# # If you have a GPU, ensure the model is on the GPU
# if torch.cuda.is_available():
#     model = model.cuda()

## Load the Model and Tokenizer from HF (after pushing)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

username = "rpdpscl"
model_id = f"{username}/gemma-2b-instruct-ft-medical-qa"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
    device_map="auto"  # Automatically maps model to available devices
)

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Test the finetuned model

In [ ]:
# Helper functions to get the responses in proper format

def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

In [ ]:
query = """\n\n Please answer with one of the options in the bracket. Write reasoning in between <analysis></analysis>.
           Write answer in between <answer></answer>. Here are the inputs:

           Q: A 35-year-old woman presents to the emergency department with shortness of breath, chest pain, and palpitations.
           She mentions she has been feeling increasingly fatigued over the last month. The patient has a history of lupus
           erythematosus. Vital signs include: pulse 120/min, temperature 38.1°C (100.6°F), blood pressure 90/60 mm Hg,
           and respiratory rate 24/min. Physical examination reveals jugular venous distension, distant heart sounds, and
           mild lower extremity edema. Which of the following diagnostic tests would be most useful in confirming the
           suspected diagnosis?

           {'A': 'Chest X-ray showing bilateral pleural effusion', 'B': 'Echocardiogram revealing pericardial effusion',
           'C': 'Electrocardiogram showing widespread ST-segment elevation', 'D': 'Complete blood count showing anemia and
           leukopenia', 'E': 'Urinalysis showing proteinuria and hematuria'}
           """

result = get_completion(query=query, model=model, tokenizer=tokenizer)
print(result)


  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  

 Please answer with one of the options in the bracket. Write reasoning in between <analysis></analysis>.
           Write answer in between <answer></answer>. Here are the inputs:

           Q: A 35-year-old woman presents to the emergency department with shortness of breath, chest pain, and palpitations.
           She mentions she has been feeling increasingly fatigued over the last month. The patient has a history of lupus
           erythematosus. Vital signs include: pulse 120/min, temperature 38.1°C (100.6°F), blood pressure 90/60 mm Hg,
           and respiratory rate 24/min. Physical examination reveals jugular venous distension, distant heart sounds, and
           mild lower extremity edema. Which of the following diagnostic tests would be most useful in confirming the
           suspected diagnosis?

           {'A': 'Chest X-ray showing bilateral pleural 

In [ ]:
print(f"{result.split('Analysis:')[-1]}")


  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  

 Please answer with one of the options in the bracket. Write reasoning in between <analysis></analysis>.
           Write answer in between <answer></answer>. Here are the inputs:

           Q: A 35-year-old woman presents to the emergency department with shortness of breath, chest pain, and palpitations.
           She mentions she has been feeling increasingly fatigued over the last month. The patient has a history of lupus
           erythematosus. Vital signs include: pulse 120/min, temperature 38.1°C (100.6°F), blood pressure 90/60 mm Hg,
           and respiratory rate 24/min. Physical examination reveals jugular venous distension, distant heart sounds, and
           mild lower extremity edema. Which of the following diagnostic tests would be most useful in confirming the
           suspected diagnosis?

           {'A': 'Chest X-ray showing bilateral pleural 